In [ ]:
"""
Z-Image LoRA Fine-tuning Script (Fixed)
基于 Flow Matching 8步扩散的完整训练实现
"""

import os
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from peft import LoraConfig, get_peft_model, PeftModel
from PIL import Image
from pathlib import Path

from tqdm import tqdm
from torchvision import transforms
import json
from typing import List, Dict, Optional

# 假设这些是你的项目导入
from utils.loader import load_from_local_dir
from zimage.pipeline import generate, calculate_shift
from config import DEFAULT_MAX_SEQUENCE_LENGTH

# 关键：设置显存分配策略，减少碎片化
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True,garbage_collection_threshold:0.6'

In [2]:
class ImageTextDataset(Dataset):
    """
    图像-文本对数据集
    支持多分辨率训练
    """
    def __init__(self, data_dir: str, resolution: int = 1024):
        self.data_dir = Path(data_dir)
        self.resolution = resolution
        
        # 收集所有图片
        self.image_paths = []
        for ext in ["*.jpg", "*.jpeg", "*.png", "*.webp"]:
            self.image_paths.extend(list(self.data_dir.glob(ext)))
        
        # 加载对应文本描述
        self.captions = {}
        for img_path in self.image_paths:
            txt_path = img_path.with_suffix(".txt")
            if txt_path.exists():
                with open(txt_path, "r", encoding="utf-8") as f:
                    self.captions[img_path] = f.read().strip()
            else:
                # 如果没有txt，使用图片名（去掉扩展名）作为提示词
                self.captions[img_path] = img_path.stem.replace("_", " ")
        
        # 图像预处理（与推理一致，归一化到[-1, 1]）
        self.transform = transforms.Compose([
            transforms.Resize(resolution, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(resolution),
            transforms.ToTensor(),  # [0, 1]
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),  # [-1, 1]
        ])
        
        print(f"加载了 {len(self.image_paths)} 张训练图片")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        try:
            image = Image.open(image_path).convert("RGB")
            image = self.transform(image)
        except Exception as e:
            print(f"加载图片失败 {image_path}: {e}")
            # 返回空白图作为fallback
            image = torch.zeros(3, self.resolution, self.resolution)
        
        caption = self.captions[image_path]
        
        return {
            "image": image,
            "caption": caption,
        }


class ZImageLoRATrainer:
    """
    Z-image LoRA 训练器
    完全复用推理代码中的 Scheduler 和组件
    """
    def __init__(
        self,
        components: Dict,
        device: str = "cuda",
        dtype: torch.dtype = torch.bfloat16,
        lora_rank: int = 64,
        lora_alpha: int = 128,
        lora_dropout: float = 0.0,
    ):
        self.device = device
        self.dtype = dtype
        
        # 解包组件（完全复用 loader.py 加载的组件）
        self.transformer = components["transformer"]
        self.vae = components["vae"]
        self.text_encoder = components["text_encoder"]
        self.tokenizer = components["tokenizer"]
        self.scheduler = components["scheduler"]  # 关键：复用 FlowMatchEulerDiscreteScheduler
        
        vae_config = self.vae.config
        self.vae_scaling_factor = getattr(vae_config, 'scaling_factor', 0.3611)
        self.vae_shift_factor = getattr(vae_config, 'shift_factor', 0.1159)
    
        print(f"✅ VAE Config - Scaling: {self.vae_scaling_factor}, Shift: {self.vae_shift_factor}")
        print(f"   Latent 目标范围: ~[-10, 10] (匹配推理)")

        # 冻结基础模型（只训练 LoRA）
        self.vae.requires_grad_(False)
        self.text_encoder.requires_grad_(False)
        self.transformer.requires_grad_(False)
        
        # 配置 LoRA
        target_modules = self._detect_target_modules()
        print(f"检测到 {len(target_modules)} 个 LoRA 目标模块")
        
        lora_config = LoraConfig(
            r=lora_rank,
            lora_alpha=lora_alpha,
            target_modules=target_modules,
            lora_dropout=lora_dropout,
            bias="none",
            init_lora_weights="gaussian",
        )

        # 应用 LoRA 到 Transformer
        self.transformer = get_peft_model(self.transformer, lora_config)
        self.transformer.to(device, dtype=dtype)
        
        # 打印可训练参数
        self.transformer.print_trainable_parameters()
        
        # 设置训练模式
        self.transformer.train()
        self.vae.eval()
        self.text_encoder.eval()
    
    def _detect_target_modules(self) -> List[str]:
        """只选择 Transformer Block 内部的 Attention 和 FFN，严格排除所有输入/输出层"""
        target_modules = []
        
        for name, module in self.transformer.named_modules():
            # 严格排除规则（包含这些关键词的都跳过）
            if any(bad in name for bad in ["all_x_embedder", "all_final_layer", "x_pad_token", "cap_pad_token", 
                                        "cap_embedder", "t_embedder", "noise_refiner", "context_refiner"]):
                continue
                
            # 只包含规则：Attention 和 FFN
            # Z-Image 使用 to_q, to_k, to_v, to_out.0 作为 attention 投影
            # 使用 w1, w2, w3 作为 FFN 层 (参见 FeedForward 类)
            # if any(good in name for good in ["to_q", "to_k", "to_v", "to_out.0", "w1", "w2", "w3"]):
            if any(good in name for good in ["w1", "w2", "w3"]):
            # if any(good in name for good in ["to_q", "to_k", "to_v", "to_out.0"]):
                if isinstance(module, torch.nn.Linear):
                    target_modules.append(name)
        
        unique_modules = list(dict.fromkeys(target_modules))
        print(f"检测到 {len(unique_modules)} 个 LoRA 目标模块（已排除所有 embedder）")
        for name in unique_modules[:5]:  # 打印前5个作为示例
            print(f"  - {name}")
        return unique_modules
    
    def encode_prompts(self, captions: List[str], max_length: int = 512) -> List[torch.Tensor]:
        """
        编码文本提示（与推理代码完全一致）
        返回 List[Tensor] 以支持变长序列
        """
        # 使用 chat template（与推理一致）
        formatted_prompts = []
        for caption in captions:
            messages = [{"role": "user", "content": caption}]
            formatted = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
                enable_thinking=True,
            )
            formatted_prompts.append(formatted)
        
        # Tokenize（固定长度填充）
        text_inputs = self.tokenizer(
            formatted_prompts,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)
        
        # 编码（冻结）
        with torch.no_grad():
            outputs = self.text_encoder(
                input_ids=text_inputs.input_ids,
                attention_mask=text_inputs.attention_mask,
                output_hidden_states=True,
            )
            # 取倒数第二层（与推理一致）
            prompt_embeds = outputs.hidden_states[-2]
        
        # 裁剪 padding，转换为 List[Tensor]
        prompt_embeds_list = []
        masks = text_inputs.attention_mask.bool()
        for i in range(len(prompt_embeds)):
            # 只保留真实 token（去掉 padding）
            valid_embeds = prompt_embeds[i][masks[i]]
            prompt_embeds_list.append(valid_embeds)
        
        return prompt_embeds_list
    
    def get_training_timesteps(self, batch_size: int, image_seq_len: int) -> torch.Tensor:
        """
        生成训练时间步（复用 scheduler 的 shift 逻辑）
        返回 timesteps 在 [0, 1] 范围内，其中 0=clean, 1=noise
        """
        # 计算 mu（与推理代码 calculate_shift 一致）
        base_seq_len = 256
        max_seq_len = 4096
        base_shift = 0.5
        max_shift = 1.15
        
        m = (max_shift - base_shift) / (max_seq_len - base_seq_len)
        b = base_shift - m * base_seq_len
        mu = image_seq_len * m + b
        
        # 生成随机时间步 [0, 1]
        # 训练时均匀采样，让模型看到各种 t
        t_raw = torch.rand(batch_size, device=self.device)
        
        # 应用 time_shift（与 scheduler 一致）
        # 使用 scheduler.time_shift 如果可用，否则手动实现
        if hasattr(self.scheduler, 'time_shift'):
            t_shifted = self.scheduler.time_shift(mu, 1.0, t_raw)
        else:
            # 手动实现（与 FlowMatchEulerDiscreteScheduler 一致）
            import math
            t_shifted = math.exp(mu) / (math.exp(mu) + (1 / t_raw - 1) ** 1.0)
        
        # 注意：在 Flow Matching 中，通常 t=0 是干净数据，t=1 是纯噪声
        # 但 Z-Image 的实现可能需要特定方向，这里保持与原来一致
        timesteps = 1.0 - t_shifted
        
        return timesteps, mu
    
    def add_noise(self, latents: torch.Tensor, noise: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        """
        Flow Matching 前向加噪：x_t = (1-t) * x_0 + t * noise
        支持 batch 处理
        """
        # t: [B] -> [B, 1, 1, 1] 用于广播
        t = t.view(-1, 1, 1, 1)
        return (1 - t) * latents + t * noise
    
    def encode_vae(self, images: torch.Tensor) -> torch.Tensor:
        images_fp32 = images.to(dtype=torch.float32)
    
        with torch.no_grad():
            # 1. VAE Encoder 输出 (raw latent, 通常是 [-20, 20] 范围)
            h = self.vae.encoder(images_fp32)
            mean, logvar = torch.chunk(h, 2, dim=1)
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            latents = mean + eps * std  # 采样
            
            # 2. 关键修复：应用与推理完全相同的预处理
            # generate.py 中: latents = (latents / scaling) + shift
            latents = (latents / self.vae_scaling_factor) + self.vae_shift_factor
            
            # 现在 latents 范围应该在 [-8, 8] 附近，与推理一致
            return latents.to(dtype=self.dtype)
    
    def compute_loss(self, latents, noise, timesteps, prompt_embeds_list):
        """
        计算 Flow Matching 损失
        latents: [B, C, H, W] - VAE 编码后的干净 latent
        noise: [B, C, H, W] - 标准高斯噪声
        timesteps: [B] - 时间步
        prompt_embeds_list: List[Tensor] - 文本编码列表
        """
        # 加噪
        noisy_latents = self.add_noise(latents, noise, timesteps)
        
        # 转换为 List 格式以匹配 Transformer 输入
        # Transformer 期望 List[[C, F, H, W]]，其中 F 是帧数（图片为1）
        latent_list = [noisy_latents[i] for i in range(latents.shape[0])]
        # 添加帧维度 [C, H, W] -> [C, 1, H, W]
        latent_list = [x.unsqueeze(1) for x in latent_list]
        
        # Transformer 前向
        # 注意：Z-Image 使用 bfloat16/float16 训练，需要 autocast
        with torch.cuda.amp.autocast(dtype=self.dtype):
            model_output = self.transformer(
                latent_list,
                timesteps,
                prompt_embeds_list,
            )[0]  # 返回 (output, {})，取第一个元素
        
        # 处理输出：model_output 是 List[Tensor]
        # 每个 Tensor 形状是 [C, F, H, W]，F=1
        if isinstance(model_output, (list, tuple)):
            # 堆叠为 [B, C, F, H, W]
            velocity_pred = torch.stack([x for x in model_output], dim=0)
        else:
            velocity_pred = model_output
        
        # 去掉帧维度 -> [B, C, H, W]
        if velocity_pred.dim() == 5:
            velocity_pred = velocity_pred.squeeze(2)
        
        # Flow Matching 目标：v = noise - x_0
        velocity_target = noise - latents
        
        # MSE 损失
        loss = F.mse_loss(velocity_pred.float(), velocity_target.float(), reduction="mean")
        
        return loss


def train_lora(
    model_path: str,
    train_data_dir: str,
    output_dir: str = "./zimage_lora_output",
    num_train_epochs: int = 10,
    batch_size: int = 1,
    learning_rate: float = 1e-4,
    gradient_accumulation_steps: int = 4,
    save_steps: int = 500,
    resolution: int = 896,
    device: str = "cuda",
    dtype: torch.dtype = torch.bfloat16,
    lora_rank: int = 64,
    lora_alpha: int = 128,
):
    """
    主训练函数
    """
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"正在加载模型从: {model_path}")
    
    # 这里需要你实际的加载函数
    components = load_from_local_dir(
        model_path,
        device=device,
        dtype=dtype,
        verbose=True,
    )
    
    
    
    # 验证 VAE 和 Transformer 维度匹配（在创建 trainer 之前）
    print("=== 维度验证 ===")
    if hasattr(components["vae"], "encoder"):
        # 测试 VAE 输出维度
        test_img = torch.zeros(1, 3, resolution, resolution).to(device=device, dtype=torch.float32)
        with torch.no_grad():
            test_out = components["vae"].encoder(test_img)
            actual_c = test_out.shape[1] // 2  # 因为 double_z=True，输出是 2*latent_channels
            
            expected_c = components["transformer"].in_channels
            
            if actual_c != expected_c:
                print(f"❌ 错误: VAE latent channels={actual_c} ≠ Transformer in_channels={expected_c}")
                return None
            else:
                print(f"✅ 维度匹配: latent_channels={actual_c}")
    
    # 初始化训练器（这时才创建 trainer）
    trainer = ZImageLoRATrainer(
        components=components,
        device=device,
        dtype=dtype,
        lora_rank=lora_rank,
        lora_alpha=lora_alpha,
    )
    
    # 准备数据集
    dataset = ImageTextDataset(train_data_dir, resolution=resolution)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        drop_last=True,
    )
    
    # 优化器（只优化 LoRA 参数）
    optimizer = AdamW(
        filter(lambda p: p.requires_grad, trainer.transformer.parameters()),
        lr=learning_rate,
        betas=(0.9, 0.999),
        weight_decay=0.01,
    )
    
    # 学习率调度
    total_steps = len(dataloader) * num_train_epochs // gradient_accumulation_steps
    lr_scheduler = CosineAnnealingWarmRestarts(
        optimizer, T_0=max(1, total_steps//10), T_mult=2, eta_min=1e-6
    )
    
    print(f"开始训练，总步数: {total_steps}，批次大小: {batch_size}，梯度累积: {gradient_accumulation_steps}")
    
    global_step = 0
    trainer.transformer.train()
    
    # 获取 scaling factor
    # vae_scaling_factor = getattr(trainer.vae.config, 'scaling_factor', 0.18215)
    
    for epoch in range(num_train_epochs):
        epoch_loss = 0.0
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_train_epochs}")
        
        optimizer.zero_grad()
        
        for step, batch in enumerate(pbar):
            images = batch["image"].to(device=device)
            captions = batch["caption"]
            
            # 1. VAE 编码（冻结）
            with torch.no_grad():
                latents = trainer.encode_vae(images)
                
                # 应用 scaling factor（与推理一致）
                latents = latents * trainer.vae_scaling_factor
                
                # 计算 image_seq_len 用于 mu（考虑 patchify）
                # latent 形状: [B, C, H, W]
                b, c, h, w = latents.shape
                # 假设使用 2x2 patch（从 all_patch_size=(2,) 推断）
                patch_size = 2
                image_seq_len = (h // patch_size) * (w // patch_size)
                
                # 文本编码（冻结）
                prompt_embeds_list = trainer.encode_prompts(captions)
            
            # 2. 生成噪声
            noise = torch.randn_like(latents)
            
            # 3. 生成时间步（复用 scheduler 逻辑）
            timesteps, mu = trainer.get_training_timesteps(
                batch_size=latents.shape[0],
                image_seq_len=image_seq_len,
            )
            
            # 4. 计算损失（带梯度累积）
            with torch.cuda.amp.autocast(dtype=dtype):
                loss = trainer.compute_loss(latents, noise, timesteps, prompt_embeds_list)
                loss = loss / gradient_accumulation_steps
            
            loss.backward()
            
            epoch_loss += loss.item() * gradient_accumulation_steps
            
            # 5. 梯度更新
            if (step + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(trainer.transformer.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                global_step += 1
                
                pbar.set_postfix({
                    "loss": f"{loss.item() * gradient_accumulation_steps:.4f}",
                    "lr": f"{lr_scheduler.get_last_lr()[0]:.2e}",
                    "mu": f"{mu:.2f}",
                })
                
                # 保存检查点
                if global_step % save_steps == 0:
                    checkpoint_dir = os.path.join(output_dir, f"checkpoint-{global_step}")
                    trainer.transformer.save_pretrained(checkpoint_dir)
                    
                    # 保存训练配置
                    config = {
                        "lora_rank": lora_rank,
                        "lora_alpha": lora_alpha,
                        "resolution": resolution,
                        "global_step": global_step,
                        "mu": mu,
                    }
                    with open(os.path.join(checkpoint_dir, "train_config.json"), "w") as f:
                        json.dump(config, f, indent=2)
                    
                    print(f"\n✅ 保存检查点到: {checkpoint_dir}")
        
        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch {epoch+1} 完成，平均损失: {avg_loss:.4f}")
    
    # 保存最终模型
    final_dir = os.path.join(output_dir, "final_lora")
    trainer.transformer.save_pretrained(final_dir)
    print(f"🎉 训练完成！LoRA 权重保存至: {final_dir}")
    
    return trainer


def load_lora_for_inference(components: Dict, lora_path: str, device: str = "cuda", merge: bool = False):
    """
    推理时加载 LoRA 权重
    
    Args:
        components: 基础模型组件
        lora_path: LoRA 权重路径
        device: 计算设备
        merge: 是否合并权重（合并后无法继续训练，但推理更快）
    """
    transformer = PeftModel.from_pretrained(
        components["transformer"],
        lora_path,
    )
    
    if merge:
        transformer = transformer.merge_and_unload()
        print("✅ LoRA 权重已合并到基础模型")
    
    components["transformer"] = transformer.to(device)
    return components

In [ ]:
MODEL_PATH = "/home/xxx/z-image"  # 你的模型路径
TRAIN_DATA_DIR = "/home/xxx/pic_spider/downloaded_images"  # 训练数据目录
OUTPUT_DIR = "./zimage_lora_output_1"

# 开始训练
trainer = train_lora(
    model_path=MODEL_PATH,
    train_data_dir=TRAIN_DATA_DIR,
    output_dir=OUTPUT_DIR,
    num_train_epochs=15,
    batch_size=1,  # 根据显存调整，建议从1开始测试
    learning_rate=1e-5,
    gradient_accumulation_steps=4,  # 有效 batch_size = 4
    save_steps=100,
    resolution=768,
    device="cuda:0",
    dtype=torch.bfloat16,
    lora_rank=64,   # 可调整为 16/32/64/128
    lora_alpha=128, # 通常设为 2*rank
)